A partir de los datos de iris:

Ejercicio 1:

- Carga los datos originales
- Utiliza PCA para reducir la dimensionalidad de los datos. No reemplaces X con el resultado, crea una nueva X_pca. Utiliza PCA(2) y otra con PCA(3)
- Utiliza Kmeans para agrupar y predecir las clases de los nuevos datos.
- Compara los resultados para PCA(2) y PCA(3) con las clases originales (y_real).
- ¿Es mejor PCA(2) o PCA(3)?

Ejercicio 2: 

- Repite el proceso con T-SNE y DBSCAN. ¿Cuál da mejores resultados?


In [89]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

iris = datasets.load_iris()
X = iris.data
# al aplicar PCA se asume que los datos de trabajo tienen una distribución gaussiana o normal.  Por tanto, aplicamos a los datos una transformación de normalización de forma que su media sea igual a 0, y su varianza=1. Para ello, usaremos la transformación Standardscaler de sklearn.
X = StandardScaler().fit_transform(X)
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [111]:
import pandas as pd
# Creamos el dataframe
df = pd.DataFrame(data=iris.data, columns=iris.get("feature_names"))


In [90]:
from sklearn.decomposition import PCA

### 2 componentes principales

In [91]:
# Establecemos el número de componentes que queremos obtener a partir de las variables que tenemos en nuestro dataFrame

pca = PCA(n_components=2)

In [92]:
# Fijamos el modelo para las variables predictoras de la muestra de entrenamiento
pca.fit(X_train)

PCA(n_components=2)

In [93]:
# Hallamos la varianza explicativa de la mejor proyección de nuestro PCA. El primer valor es la varianza explicativa de nuestro modelo con la muestra de entrenamiento
pca.explained_variance_ratio_

array([0.71537436, 0.24083209])

In [94]:
# transformamos nuestras variables predictoras según nuestro modelo en dos componentes principales
# se obtiene el valor que toma cada componente para cada observación en función de las variables originales  

X_pca_2_train = pca.transform(X_train)

In [108]:
import matplotlib.pyplot as plt

#graficamos en 2 Dimensiones, tomando los 2 primeros componentes principales
Xax=X_pca_2_train[:,0]
Yax=X_pca_2_train[:,1]
labels=X
cdict={0:'red',1:'green'}
alpha={0:.3, 1:.5}
fig,ax=plt.subplots(figsize=(7,5))
fig.patch.set_facecolor('white')
for l in np.unique(labels):
    ix=np.where(labels==l)
    ax.scatter(Xax[ix],Yax[ix],c=cdict[l],label=labl[l],s=40,marker=marker[l],alpha=alpha[l])
 
plt.xlabel("First Principal Component",fontsize=14)
plt.ylabel("Second Principal Component",fontsize=14)
plt.legend()
plt.show()


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [95]:
# Fijamos el modelo para las variables predictoras de la muestra de test
pca.fit(X_test)

PCA(n_components=2)

In [96]:
# Hallamos la varianza explicativa de la mejor proyección de nuestro PCA. El primer valor es la varianza explicativa de nuestro modelo con la muestra de test
pca.explained_variance_ratio_

array([0.7900007 , 0.17758282])

### 3 componentes principales

In [97]:
pca = PCA(n_components=3)

In [98]:
pca.fit(X_train)

PCA(n_components=3)

In [99]:
# No se observa incremento de varianza explicativa respecto a 2 componentes así que con dos componentes principales es suficiente 
pca.explained_variance_ratio_

array([0.71537436, 0.24083209, 0.03901736])

In [100]:
  X_pca_3_train = pca.transform(X_train)

In [101]:
pca.fit(X_test)

PCA(n_components=3)

In [102]:
pca.explained_variance_ratio_

array([0.7900007 , 0.17758282, 0.02604515])

### 4. Kmeans

In [103]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=2, random_state=0).fit(X_pca_2_train)
print("\nkmeans.labels_:", kmeans.labels_)
y_predict = kmeans.fit_predict(X_pca_2_train)
print("predict:", y_predict)
clusters = kmeans.cluster_centers_
print("kmeans.cluster_centers_:\n", clusters)


kmeans.labels_: [1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0
 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0
 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0]
predict: [1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0
 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0
 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0]
kmeans.cluster_centers_:
 [[ 1.10751085  0.14710663]
 [-2.21502169 -0.29421326]]


In [104]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=2, random_state=0).fit(X_pca_3_train)
print("\nkmeans.labels_:", kmeans.labels_)
y_predict = kmeans.fit_predict(X_pca_3_train)
print("predict:", y_predict)
clusters = kmeans.cluster_centers_
print("kmeans.cluster_centers_:\n", clusters)


kmeans.labels_: [1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0
 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0
 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0]
predict: [1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0 1 0
 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0
 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0]
kmeans.cluster_centers_:
 [[ 1.10751085  0.14710663  0.01949137]
 [-2.21502169 -0.29421326 -0.03898273]]


### Comparar el modelo de PCA con las clases originales

In [ ]:
pca = PCA(n_components=2)

In [ ]:
# fijamos el modelo para todos los valores
pca.fit(X)

In [ ]:
# varianza explicada para todos los valores
pca.explained_variance_ratio_

In [ ]:
pca = PCA(n_components=3)

In [ ]:
pca.fit(X)

In [ ]:
pca.explained_variance_ratio_